In [14]:
# !pip install -U langchain-openai

In [3]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os 

# Load environment variables
load_dotenv()

model = AzureChatOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_CHAT_API_VERSION"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    streaming=True
)

In [4]:
import pandas as pd 

df=pd.read_csv(r'C:\Users\AS9969\Desktop\Ashish Work Folder\Saswats Project\One Zebra JIRA 2025-10-24T07_39_57-0400 1.csv')

In [17]:
# df[['Issue key', 'Issue id', 'Summary', 'Issue Type','Component/s','Component/s.1', 'Component/s.2','Custom field (Root Cause)', 'Custom field (RCA Subcategory)']]

In [5]:
def language_translator_summary_cleaner(col):
    prompt = """You are a multilingual data-cleaning assistant.  
                Your task is to process text values that may contain a mix of English, French, or other languages along with unnecessary special characters or symbols.

                For each input text:
                1. Remove all unnecessary special characters, emojis, and noise while keeping valid punctuation.  
                2. Detect if the text contains mixed languages (especially English and French).  
                3. Translate the entire text into fluent, grammatically correct English.  
                4. Return only the cleaned and translated English text — no explanations or formatting.

                Example:
                Input: "Bonjour! This produit est génial!!! 🤩"
                Output: "Hello! This product is great.
            """
    cleaned_col = []
    for i in col:
        try:
            if not isinstance(i, str) or not i.strip():
                cleaned_col.append("Blank")
                continue

            result = model.invoke([
                {"role": "system", "content": prompt},
                {"role": "user", "content": i}
            ])

            response_text = getattr(result, "content", str(result))
            cleaned_col.append(response_text.strip())
        except Exception as e:
            print("Error:", e)
            cleaned_col.append("Blank")
    return cleaned_col 


language_translator_summary_cleaner(df['Summary'])

Error: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': True, 'detected': True}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}


KeyboardInterrupt: 

In [3]:
import asyncio,nest_asyncio
nest_asyncio.apply()

async def process_text(i):
    prompt = """You are a multilingual data-cleaning assistant.  
                Your task is to process text values that may contain a mix of English, French, or other languages along with unnecessary special characters or symbols.

                For each input text:
                1. Remove all unnecessary special characters, emojis, and noise while keeping valid punctuation.  
                2. Detect if the text contains mixed languages (especially English and French).  
                3. Translate the entire text into fluent, grammatically correct English.  
                4. Return only the cleaned and translated English text — no explanations or formatting.

                Example:
                Input: "Bonjour! This produit est génial!!! 🤩"
                Output: "Hello! This product is great.
            """
    try:
        if not isinstance(i, str) or not i.strip():
            return "Blank"
        result = await model.ainvoke([
            {"role": "system", "content": prompt},
            {"role": "user", "content": i}
        ])
        return getattr(result, "content", str(result)).strip()
    except Exception:
        return "Blank"

def language_translator_summary_cleaner_async(col):
    async def _inner():
        tasks = [process_text(i) for i in col]
        results = await asyncio.gather(*tasks)
    return asyncio.run()

# run
cleaned = language_translator_summary_cleaner_async(df["Summary"])

TypeError: _patch_asyncio.<locals>.run() missing 1 required positional argument: 'main'

In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

# --- Async text processor ---
async def process_text(i, model):
    \
    try:
        if not isinstance(i, str) or not i.strip():
            return "Blank"

        result = await model.ainvoke([
            {"role": "system", "content": prompt},
            {"role": "user", "content": i}
        ])
        return getattr(result, "content", str(result)).strip()

    except Exception as e:
        print("Error:", e)
        return "Blank"


# --- Async wrapper function ---
async def _language_translator_summary_cleaner_async(col, model):
    tasks = [process_text(i, model) for i in col]
    results = await asyncio.gather(*tasks)
    return results


def language_translator_summary_cleaner_async(col, model):
    """Safe wrapper for both notebook and script environments"""
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running(): # e.g., Jupyter
            return loop.run_until_complete(_language_translator_summary_cleaner_async(col, model))
        else:
            return asyncio.run(_language_translator_summary_cleaner_async(col, model))
    except RuntimeError: # fallback for environments already patched
        return asyncio.run(_language_translator_summary_cleaner_async(col, model))


# --- Run ---
cleaned = language_translator_summary_cleaner_async(df["Summary"], model)

ValueError: too many file descriptors in select()

: 

In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

async def process_text(i, model, sem):
    prompt = """You are a multilingual data-cleaning assistant.
    Your task is to process text values that may contain a mix of English, French, or other languages along with unnecessary special characters or symbols.

    For each input text:
    1. Remove all unnecessary special characters, emojis, and noise while keeping valid punctuation.
    2. Detect if the text contains mixed languages (especially English and French).
    3. Translate the entire text into fluent, grammatically correct English.
    4. Return only the cleaned and translated English text — no explanations or formatting.

    Example:
    Input: "Bonjour! This produit est génial!!! 🤩"
    Output: "Hello! This product is great."
    """
    async with sem: # 👈 concurrency limiter
        try:
            if not isinstance(i, str) or not i.strip():
                return "Blank"
            result = await model.invoke([
                {"role": "system", "content": prompt},
                {"role": "user", "content": i}
            ])
            return getattr(result, "content", str(result)).strip()
        except Exception:
            return "Blank"

async def _language_translator_summary_cleaner_async(col, model, concurrency=10):
    sem = asyncio.Semaphore(concurrency)
    tasks = [process_text(i, model, sem) for i in col]
    return await asyncio.gather(*tasks)

def language_translator_summary_cleaner_async(col, model):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(_language_translator_summary_cleaner_async(col, model, concurrency=10))

# run
cleaned = language_translator_summary_cleaner_async(df["Summary"], model)